# INTRODUCTION

This project contains the files for the first project in the *Udacity Deep Reinforcement Learning Nanodegree*. It places an agent in an environment where the goal of the agent is to maximize its rewards given a fixed limit of 300 actions.

The environment consists of a walled area with yellow and blue bananas placed at random in the playing field. The goal for the agent is to collect as many **yellow bananas (+1 reward)** as possible while avoiding **blue bananas (-1 reward)**. 

The actions available to the agent are:
* move forwards
* move backwards
* turn left
* turn right

At first the agent's movements are random, but over time it learns which actions to take to maximize its rewards.

The goal of the project is to average 13 points over the previous 100 episodes. If the agent is able to do this, the task is considered "solved" and the training ends.

# IMPLEMENTATION

| Time | episodes |     structure    |  update  |
|:----:|:--------:|:----------------:|:--------:|
|      |          |                  |          |
|  638 |    559   |  512 / 512 / 256 |  128 / 6 |
|  348 |    434   |  256 / 256 / 256 |  128 / 6 |
|  526 |    567   |  256 / 256 / 64  |  32 / 2  |
|  343 |    563   |  256 / 256 / 64  | 128 / 12 |
|  393 |    599   |  256 / 256 / 64  | 256 / 12 |
|  964 |    513   | 2048 / 512 / 128 |  128 / 6 |
|  369 |    584   |  256 / 256 / 64  | 128 / 12 |
|  370 |    484   |  256 / 256 / 64  |  128 / 6 |
|  320 |    453   |  128 / 128 / 64  |  128 / 6 |
|  310 |    455   |  128 / 128 / 64  |  128 / 6 |
|  343 |    484   |   64 / 64 / 64   |  128 / 6 |
|  294 |    452   |   64 / 64 / 64   |  128 / 6 |